In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("mempool.csv")

In [3]:
df.head(10)

,tx_id,fee,weight,parents
0,2e3da8fbc1eaca8ed9b7c2db9e6545d8ccac3c67deadee...,452,1620,NaN
1,79c51c9d4124c5cbb37a85263748dcf44e182dff83561f...,682,1136,6eb38fad135e38a93cb47a15a5f953cbc0563fd84bf1ab...
2,b0ef627c8dc2a706475d33d7712209ec779f7a8302aaea...,226,900,NaN
3,8c25f9be93990b96e8bc363778d6debee6867c7d73cefa...,258,1032,NaN
4,edc82751537be6afcb21c0229d8d20916d8e742a363ab2...,452,1616,NaN
5,384718a612cc6b220a8eeaf36f9d179e70fe233e2e9068...,452,1616,edc82751537be6afcb21c0229d8d20916d8e742a363ab2...
6,d506243d15088e94cf5300c55c67889d61f798750f6c59...,285,1132,NaN
7,857909bcca1f5e76d2eb78d60ecf14d6d6dcfd3691c950...,285,1132,NaN
8,e50935ba818134e052de9e65398faa5a28fde36a885367...,285,1136,NaN
9,8ce84562ac4aab9c1ec15ac0c76cf0ef5ae5f9a945d717...,285,1136,NaN


In [4]:
df.shape

(5214, 4)

In [5]:
df.dtypes

tx_id       object
fee          int64
weight       int64
parents     object
dtype: object

In [6]:
df.describe()

,fee,weight
count,5214.000000,5214.000000
mean,1435.671461,2000.690449
std,5687.415106,7013.491561
min,110.000000,440.000000
25%,336.000000,888.000000
50%,592.500000,1132.000000
75%,1008.000000,1136.000000
max,201700.000000,295164.000000


In [7]:
def sort_trans(df, maxfee, minweight):
    df = df.sort_values([maxfee, minweight], ascending=[False, True]).reset_index(drop=True)
    return df

In [8]:
def check_weight(x):
    if min_wt + x['weight'] <= high_wt:
        return True
    else:
        return False

In [9]:
def check_list(x):
    if str(x) in final_txids:
        return True
    else:
        return False

In [10]:
def check_parent(x):
    if str(x[3]) != "nan":
        parent_list = str(x[3]).split(";")
        for i in parent_list:
            if(check_list(i)):
                continue
            else:
                txnindex = df[df['tx_id'] == i].index.item()
                k = df.loc[txnindex]
                check_add_txn(k)

In [11]:
def add_to_block(x):
    global min_wt
    txnID = x[0]
    weight = x[2]
    min_wt += weight
    final_txids.append(txnID)

In [12]:
def check_add_txn(x):
    if(check_weight(x)):
        if(not check_list(x)):
            check_parent(x)
            if(check_weight(x)):
                add_to_block(x)

In [13]:
def Main(df):
    s_trans = sort_trans(df, "fee", "weight")
    for i in range(len(s_trans)):
        txnVar =  s_trans.loc[i]
        check_add_txn(txnVar)

In [14]:
def write_to_file(fin_list):
    file = open("block.txt","a")
    for i in fin_list:
        file.write(str(i) + '\n')
    file.close()

In [15]:
high_wt = 4000000
min_wt = 0
final_txids = []
data = df
Main(data)

In [16]:
write_to_file(final_txids)